In [1]:
!pip install --upgrade --quiet langchain langchain-community langchain-groq neo4j


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("API_KEY")

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [7]:
from langchain_community.graphs import Neo4jGraph

graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

/var/folders/m5/7ngymmln16d_rwqzf8f8mhvc0000gn/T/ipykernel_6048/2997477937.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


In [8]:
graph

In [9]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=API_KEY, model_name="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x123c38050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x123c38d70>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# This below 4-5 cells are just an example of how a normal text document could be converted into a graph document using a LLMGraphTransformer

from langchain_core.documents import Document

text = """
    Elon Reeve Musk (born June 28, 1971) is an international businessman and entrepreneur known for his leadership of Tesla, 
    SpaceX, X (formerly Twitter), and xAI . Musk has been the wealthiest person in the world since 2021; as of October 2025, Forbes estimates his 
    net worth to be US$500 billion.

    Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada; he had obtained Canadian citizenship at birth through his 
    Canadian-born mother. He received bachelor's degrees in 1997 from the University of Pennsylvania in Philadelphia, United States, 
    before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded 
    X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also became an American citizen.
    """

documents = [Document(page_content=text)]

documents

[Document(metadata={}, page_content="\n    Elon Reeve Musk (born June 28, 1971) is an international businessman and entrepreneur known for his leadership of Tesla, \n    SpaceX, X (formerly Twitter), and xAI . Musk has been the wealthiest person in the world since 2021; as of October 2025, Forbes estimates his \n    net worth to be US$500 billion.\n\n    Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada; he had obtained Canadian citizenship at birth through his \n    Canadian-born mother. He received bachelor's degrees in 1997 from the University of Pennsylvania in Philadelphia, United States, \n    before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded \n    X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also became an American citizen.\n    ")]

In [13]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

# Converting our text document into graph documents

graph_documents = llm_transformer.convert_to_graph_documents(documents)

graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Tesla', type='Company', properties={}), Node(id='Spacex', type='Company', properties={}), Node(id='X', type='Company', properties={}), Node(id='Xai', type='Company', properties={}), Node(id='University Of Pennsylvania', type='University', properties={}), Node(id='Zip2', type='Company', properties={}), Node(id='X.Com', type='Company', properties={}), Node(id='Paypal', type='Company', properties={}), Node(id='Ebay', type='Company', properties={})], relationships=[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla', type='Company', properties={}), type='CEO', properties={}), Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Company', properties={}), type='FOUNDER', properties={}), Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X', type='Company', pr

In [14]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Tesla', type='Company', properties={}),
 Node(id='Spacex', type='Company', properties={}),
 Node(id='X', type='Company', properties={}),
 Node(id='Xai', type='Company', properties={}),
 Node(id='University Of Pennsylvania', type='University', properties={}),
 Node(id='Zip2', type='Company', properties={}),
 Node(id='X.Com', type='Company', properties={}),
 Node(id='Paypal', type='Company', properties={}),
 Node(id='Ebay', type='Company', properties={})]

In [15]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla', type='Company', properties={}), type='CEO', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Company', properties={}), type='FOUNDER', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X', type='Company', properties={}), type='OWNER', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Xai', type='Company', properties={}), type='FOUNDER', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='University Of Pennsylvania', type='University', properties={}), type='ATTENDED', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Zip2', type='Company', properties={}), type='CO-FOUNDER'

In [16]:
# Load the movie dataset

movie_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH(director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [17]:
graph

In [18]:
graph.query(movie_query)

[]

In [19]:
print(graph.schema)

Node properties:
Person {born: INTEGER, name: STRING}
Role {appointed: INTEGER, jobtitle: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:WORKS_AS]->(:Role)
(:Person)-[:SUPERVISOR]->(:Person)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [21]:
from langchain.chains import GraphCypherQAChain

# What GraphCypherQAChain does is based on the user query it creates a Cypher query langauge to hit the database that accesses the answer to the user query

chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, allow_dangerous_requests=True, verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x106d1e900>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x123c38050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x123c38d70>, model_name='gemma2-9b-it', model

In [22]:
response = chain.invoke({"query":"List the actors in Toy Story?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: "Toy Story"})<-[:ACTED_IN]-(p:Person)RETURN p.name

Full Context:
[{'p.name': 'Jim Varney'}, {'p.name': 'Tim Allen'}, {'p.name': 'Tom Hanks'}, {'p.name': 'Don Rickles'}]

> Finished chain.


{'query': 'List the actors in Toy Story?',
 'result': 'Jim Varney, Tim Allen, Tom Hanks, Don Rickles \n'}